In [5]:
import pandas as pd

In [6]:
# Read the Excel workbook
df = pd.read_excel('data/2018-usda-food-environment-atlas-dataset.xls')

ImportError: Install xlrd >= 0.9.0 for Excel support

In [ ]:
df.head()

In [ ]:
# Load Access, Stores, Assistance, Insecurity, Local, Health, Restaurants, Socioeconomic data sheets as dataframes 

access = pd.read_excel('data/2018-usda-food-environment-atlas-dataset.xls', 'ACCESS')
stores = pd.read_excel('data/2018-usda-food-environment-atlas-dataset.xls', 'STORES')
assistance = pd.read_excel('data/2018-usda-food-environment-atlas-dataset.xls', 'ASSISTANCE')
insecurity = pd.read_excel('data/2018-usda-food-environment-atlas-dataset.xls', 'INSECURITY')
local = pd.read_excel('data/2018-usda-food-environment-atlas-dataset.xls', 'LOCAL')
health = pd.read_excel('data/2018-usda-food-environment-atlas-dataset.xls', 'HEALTH')
restaurants = pd.read_excel('data/2018-usda-food-environment-atlas-dataset.xls', 'RESTAURANTS')
socioeconomic = pd.read_excel('data/2018-usda-food-environment-atlas-dataset.xls', 'SOCIOECONOMIC')
prices_taxes = pd.read_excel('data/2018-usda-food-environment-atlas-dataset.xls', 'PRICES_TAXES')

In [ ]:
import seaborn as sns
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [ ]:
restaurants.columns

In [ ]:
restaurant_cols = ['FIPS', 'State', 'County', 'FFRPTH14', 'FSRPTH14']
stores_cols = ['FIPS', 'State', 'County', 'GROCPTH14', 'SUPERCPTH14', 'CONVSPTH14', 'SPECSPTH14', 'SNAPSPTH16']
access_cols = ['FIPS', 'State', 'County', 'PCT_LACCESS_POP15', 'PCT_LACCESS_LOWI15', 'PCT_LACCESS_HHNV15', 'PCT_LACCESS_SNAP15', 'PCT_LACCESS_CHILD15', 'PCT_LACCESS_SENIORS15', 'PCT_LACCESS_WHITE15', 'PCT_LACCESS_BLACK15', 'PCT_LACCESS_HISP15', 'PCT_LACCESS_NHASIAN15', 'PCT_LACCESS_NHNA15', 'PCT_LACCESS_NHPI15', 'PCT_LACCESS_MULTIR15']
assistance_cols = ['FIPS', 'State', 'County', 'PCT_SNAP16', 'PCT_NSLP15', 'PCT_SBP15', 'PCT_SFSP15', 'PCT_WIC15']
prices_taxes_cols = ['FIPS', 'State', 'County', 'SODATAX_STORES14', 'SODATAX_VENDM14', 'CHIPSTAX_STORES14', 'CHIPSTAX_VENDM14', 'FOOD_TAX14']
local_cols = ['FIPS', 'State', 'County', 'FMRKTPTH16', 'PCT_FMRKT_SNAP16', 'PCT_FMRKT_WIC16', 'PCT_FMRKT_WICCASH16', 'PCT_FMRKT_SFMNP16', 'PCT_FMRKT_CREDIT16', 'PCT_FMRKT_FRVEG16', 'PCT_FMRKT_ANMLPROD16', 'PCT_FMRKT_BAKED16', 'PCT_FMRKT_OTHERFOOD16', 'FOODHUB16']
health_cols = ['FIPS', 'State', 'County', 'PCT_DIABETES_ADULTS13', 'PCT_OBESE_ADULTS13', 'PCT_HSPA15', 'RECFACPTH14']
socioeconomic_cols = ['FIPS', 'State', 'County', 'PCT_NHWHITE10', 'PCT_NHBLACK10', 'PCT_HISP10', 'PCT_NHASIAN10', 'PCT_NHNA10', 'PCT_NHPI10', 'PCT_65OLDER10', 'PCT_18YOUNGER10', 'MEDHHINC15', 'POVRATE15', 'METRO13']


In [ ]:
features_new = stores_cols + assistance_cols + local_cols + health_cols + restaurant_cols + socioeconomic_cols + prices_taxes_cols + access_cols
prediction_features = list(set(features_new) - set(['FIPS', 'State', 'County', 'PCT_DIABETES_ADULTS13', 'PCT_OBESE_ADULTS13']))
target = 'PCT_OBESE_ADULTS13'




In [ ]:
restaurants.columns

In [ ]:

dfs = [stores, assistance, insecurity, local, health, restaurants, socioeconomic, prices_taxes]
for df in dfs:
    df.drop(columns=['State', 'County'], axis=1, inplace=True)
    df.set_index('FIPS', inplace=True)



In [ ]:
stores.head()

In [ ]:
access.set_index('FIPS', drop=False, inplace=True)

In [ ]:
access.head()

In [ ]:
stores.head()

In [ ]:
# Combine all sheets into one dataframe by joining on FIPS col.

#master_df = []
#for df in dfs:
#    master_df = access.join(df, on="FIPS")
# In [37]: df1.merge(df2, right_index=True, left_on='v2', how='outer')

master_df = access.join(dfs)

In [ ]:
access.head()

In [ ]:
master_df['County'].dtype

In [ ]:
master_df.head()

In [ ]:
master_df.shape

In [ ]:
df = master_df[list(set(features_new))]
df.shape

In [ ]:
df.tail()

In [ ]:
df.isnull().sum()

In [ ]:
df = df.dropna()
df.isnull().sum()

In [ ]:
# make sure metro is boolean
df['METRO13'] = df['METRO13'].astype('bool')

In [ ]:
print(df.dtypes)

In [ ]:
df.to_csv('data/df_for_webapp.csv', index=False)

In [ ]:
df_test = df

In [ ]:
df_test['County'] = (df_test['County'].values)

In [ ]:
df_test['County'].values

In [ ]:
df_test.head()

In [ ]:
print ((df[df['County'] == 'Orange'].County.values))

In [ ]:
for county in df['County'].values:
    str(county)

df['County'].values

In [ ]:
df['State'] = df['State'].astype('category')

In [ ]:
# df.to_csv('data/df_for_webapp.csv', index=False)